# Agents

This notebook covers basic functionality related to agents. For more in detail documentation, see:

- [Agent Documentation](https://python.langchain.com/docs/modules/agents/)

- [Agent Types](https://python.langchain.com/docs/modules/agents/agent_types/)

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [2]:
import logging
from dotenv import load_dotenv

In [3]:
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

load_dotenv()

True

## Prompt

In [4]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

In [5]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

## LLM

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

## Tools

In [7]:
search = TavilySearchResults()
tools = [search]

## Agent

In [8]:
agent = create_openai_functions_agent(llm, tools, prompt)

In [9]:
result = agent.invoke({"input": "what's the weather in SF?", "intermediate_steps": []})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [10]:
result.tool

'tavily_search_results_json'

In [11]:
result.tool_input

{'query': 'weather in San Francisco'}

In [12]:
result

AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})])

## Agent Executor

In [13]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [14]:
agent_executor.invoke({"input": "what is the weather in sf?"})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'input': 'what is the weather in sf?',
 'output': "I'm sorry, but I couldn't find the current weather in San Francisco. However, you can check websites like Weather.com or AccuWeather for the most up-to-date weather information."}

## Streaming

In [15]:
for step in agent_executor.stream({"input": "what is the weather in sf?"}):
    print(step)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'actions': [AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})])], 'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})]}
{'steps': [AgentStep(action=AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}})]), observation=[{'url': '

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'output': "I'm sorry, but I couldn't find the current weather in San Francisco. However, you can check websites like Weather.com or AccuWeather for the most up-to-date weather information.", 'messages': [AIMessage(content="I'm sorry, but I couldn't find the current weather in San Francisco. However, you can check websites like Weather.com or AccuWeather for the most up-to-date weather information.")]}
